In [1]:
spark

In [1]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

In [1]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover
import jieba

In [49]:
df = spark.read.load(path,format="csv", sep="{", inferSchema="true", header="true")

In [50]:
df2 = df.withColumnRenamed("生技醫療類指數","tourism") \
    .withColumnRenamed("觀光類指數","bio")\
    .withColumnRenamed("網站","website")\
    .withColumnRenamed("關鍵字","keyword")\
    .withColumnRenamed("網址","url")\
    .withColumnRenamed("新聞標題","title")\
    .withColumnRenamed("內容","content")\
    .withColumnRenamed("預計影響日期","effect_time")

In [51]:
df_ = df2.select(['content'])

In [56]:
df_try.write.parquet("ml_test/news.parquet")

In [57]:
df = spark.read.parquet("ml_test/news.parquet")

In [58]:
df.createOrReplaceTempView("news")

In [59]:
stop_words=open('./stopwords.txt', 'r', encoding='utf-8').readlines()
stop_words = [line.strip() for line in stop_words]

In [60]:
spark.sparkContext.addFile('userdict.txt')
spark.sparkContext.addFile('dict.txt.big')

In [61]:
def seg_list(x):
    if not jieba.dt.initialized:
        jieba.set_dictionary('dict.txt.big')
        jieba.load_userdict('userdict.txt')
    s=jieba.lcut(x, cut_all=False)
    s=[x for x in s if len(x)>1 and x not in stop_words]
    
    return s            
seg_udf = udf(seg_list, ArrayType(StringType()))

In [62]:
df = spark.sql("select * from news ").cache()

In [63]:
df = df.withColumn('tokens',seg_udf(df['content']))

In [68]:
df_final = df.select('tokens')

In [71]:
df_final.write.parquet("ml_test/news_jieba.parquet")

In [4]:
df = spark.read.parquet("ml_test/news_jieba.parquet")

In [5]:
df1 = df.limit(50000)

In [ ]:
import datetime
print(datetime.datetime.now())
word2Vec = Word2Vec(vectorSize=250, minCount=5, inputCol="tokens", outputCol="result")
#word2Vec = Word2Vec(vectorSize=250, minCount=5, inputCol="tokens", outputCol="result", windowSize = 10, numPartitions=10,maxIter=10)
model = word2Vec.fit(df1)

print(datetime.datetime.now())

In [7]:
model.save("ml_test/word5vec-model")

In [33]:
model.findSynonyms('口罩', 5).collect() #10000

[Row(word='配送', similarity=0.613333523273468),
 Row(word='買不到', similarity=0.5987043976783752),
 Row(word='佛心', similarity=0.5964447259902954),
 Row(word='發售', similarity=0.5891783833503723),
 Row(word='買到', similarity=0.5793681740760803)]

In [36]:
model.findSynonyms('口罩', 5).collect() #30000

[Row(word='外科', similarity=0.48661717772483826),
 Row(word='手套', similarity=0.47157344222068787),
 Row(word='單次', similarity=0.4420098662376404),
 Row(word='醫用', similarity=0.44008582830429077),
 Row(word='乳膠手套', similarity=0.43241336941719055)]

In [7]:
model.findSynonyms('口罩', 5).collect() #35000

[Row(word='戴上', similarity=0.524684488773346),
 Row(word='醫用', similarity=0.4781174659729004),
 Row(word='港民', similarity=0.4763379991054535),
 Row(word='配戴', similarity=0.47589749097824097),
 Row(word='外科', similarity=0.47426867485046387)]

In [5]:
model.findSynonyms('口罩', 5).collect() #40000

[Row(word='佩戴', similarity=0.5079847574234009),
 Row(word='社交距離', similarity=0.5024327635765076),
 Row(word='配戴', similarity=0.47091391682624817),
 Row(word='勤洗手', similarity=0.4567772448062897),
 Row(word='戴上', similarity=0.45477065443992615)]